In [ ]:
using ZipFile, SExpressions, HerbGrammar

In [ ]:
out = download("https://github.com/thelmuth/Clojush/archive/refs/tags/psb2-v1.0.zip")

r = ZipFile.Reader(out);

In [ ]:
r = ZipFile.Reader(out)
instructions = filter(x -> contains(x.name, "instruction") && contains(x.name, ".clj"), r.files)
instruction_set = Dict()

for i in instructions
    name = split(i.name, "/")[end]
    contents = String(read(i))
    contents = replace(
        contents,
        r"\#\{(?<setitems>.+)\}" => s"(hash-set \g<setitems>)", # Replace #{...} with (hash-set ...)
        r"#\_" => "", # Replace #_ (comment) with nothing
        r"#\(" => "(ANONYMOUS ", # Replace #"..." (regex in Clojure) with r"..." (regex in Julia)
        r"#" => "r",
        "\\" => "\\\\",
        ";(update-in [:gtm :position] inc)" => "",
    )

    try
        s = SExpressions.parseall(contents)
        instruction_set[name] = s.cdr
    catch err
        println(">>> ERROR parsing $name: $err")
    end
end

In [ ]:
function make_instruction_table(instruction_set)
    """
    Recursively get the .cdr of each instruction in the instruction set.
    """
    instructions = Dict()
    if isa(instruction_set, Nil)
        return instructions
    end
    if instruction_set.car.car == Symbol("define-registered")
        instructions[instruction_set.car.cdr.car] = instruction_set.car.cdr.cdr
        instructions = merge(instructions, make_instruction_table(instruction_set.cdr))
    else
        instructions = merge(instructions, make_instruction_table(instruction_set.cdr))
    end

    return instructions
end

In [ ]:
flattened_instruction_sets = Dict(k => make_instruction_table(v) for (k, v) in instruction_set)

In [ ]:
for (k, v) in flattened_instruction_sets
    for (k2, v2) in v
        println("$k.$k2")
    end
end

In [ ]:
# Clojure code:
# (^ (:stack-types (:char :boolean)) (fn (state) (if (not (empty? (:char state))) (let (item (stack-ref :char 0 state)) (->> (pop-item :char state) (push-item (or (= item \\newline) (= item \\space) (= item \\tab)) :boolean))) state)))

# Corresponding Julia code:
function char_iswhitespace(state)
    if !isempty(state[:char])
        item = state[:char][1]
        state[:char] = state[:char][2:end]
        state[:boolean] = [item == '\n' || item == ' ' || item == '\t']
    end
    return state
end

# Clojure code:
# (^ (:stack-types (:char)) (fn (state) (if (not (empty? (:char state))) (let (cha (stack-ref :char 0 state)) (->> (pop-item :char state) (push-item (if (and (>= (int cha) 97) (<= (int cha) 122)) (char (- (int cha) 32)) cha) :char))) state)))

# Corresponding Julia code:
function char_uppercase(state)
    if !isempty(state[:char])
        cha = state[:char][1]
        state[:char] = state[:char][2:end]
        state[:char] = [isascii(cha) && islowercase(cha) ? uppercase(cha) : cha]
    end
    return state
end

# Clojure code:
# (^ (:stack-types (:char :integer)) (fn (state) (if (not (empty? (:integer state))) (let (item (stack-ref :integer 0 state)) (->> (pop-item :integer state) (push-item (char (mod item 128)) :char))) state)))

# Corresponding Julia code:
function char_frominteger(state)
    if !isempty(state[:integer])
        item = state[:integer][1]
        state[:integer] = state[:integer][2:end]
        state[:char] = [Char(mod(item, 128))]
    end
    return state
end

# Clojure code:
# (^ (:stack-types (:char :string)) (fn (state) (if (empty? (:string state)) state (loop (char-list (reverse (top-item :string state)) loop-state (pop-item :string state)) (if (empty? char-list) loop-state (recur (rest char-list) (push-item (first char-list) :char loop-state)))))))

# Corresponding Julia code:
function char_allfromstring(state)
    if isempty(state[:string])
        return state
    end
    loop_state = state
    char_list = reverse(state[:string][1])
    state[:string] = state[:string][2:end]
    while !isempty(char_list)
        loop_state = push_item!(loop_state, char_list[1], :char)
        char_list = char_list[2:end]
    end
    return loop_state
end

# Clojure code:
# (^ (:stack-types (:char :float)) (fn (state) (if (not (empty? (:float state))) (let (item (stack-ref :float 0 state)) (->> (pop-item :float state) (push-item (char (mod (long item) 128)) :char))) state)))

# Corresponding Julia code:
function char_fromfloat(state)
    if !isempty(state[:float])
        item = state[:float][1]
        state[:float] = state[:float][2:end]
        state[:char] = [Char(mod(Int64(item), 128))]
    end
    return state
end

# Clojure code:
# (^ (:stack-types (:char)) (fn (state) (if (not (empty? (:char state))) (let (cha (stack-ref :char 0 state)) (->> (pop-item :char state) (push-item (if (and (>= (int cha) 65) (<= (int cha) 90)) (char (+ (int cha) 32)) cha) :char))) state)))

# Corresponding Julia code:
function char_lowercase(state)
    if !isempty(state[:char])
        cha = state[:char][1]
        state[:char] = state[:char][2:end]
        state[:char] = [isascii(cha) && isuppercase(cha) ? lowercase(cha) : cha]
    end
    return state
end

# Clojure code:
# (^ (:stack-types (:char :integer)) (fn (state) (if (not (empty? (:integer state))) (let (item (stack-ref :integer 0 state)) (->> (pop-item :integer state) (push-item (char (mod item 256)) :char))) state)))

# Corresponding Julia code:
function char_frominteger(state)
    if !isempty(state[:integer])
        item = state[:integer][1]
        state[:integer] = state[:integer][2:end]
        state[:char] = [Char(mod(item, 256))]
    end
    return state
end

# Clojure code:
# (^ (:stack-types (:char :boolean)) (fn (state) (if (not (empty? (:char state))) (let (item (stack-ref :char 0 state)) (->> (pop-item :char state) (push-item (Character/isLetter item) :boolean))) state)))

# Corresponding Julia code:
function char_isletter(state)
    if !isempty(state[:char])
        item = state[:char][1]
        state[:char] = state[:char][2:end]
        state[:boolean] = [isletter(item)]
    end
    return state
end

# Clojure code for char_isdigit:
# (^ (:stack-types (:char :boolean)) (fn (state) (if (not (empty? (:char state))) (let (item (stack-ref :char 0 state)) (->> (pop-item :char state) (push-item (Character/isDigit item) :boolean))) state)))

# Corresponding Julia code:
function char_isdigit(state)
    if !isempty(state[:char])
        item = state[:char][1]
        state[:char] = state[:char][2:end]
        state[:boolean] = [isdigit(item)]
    end
    return state
end


g = @cfgrammar begin
    State = char_allfromstring | char_fromfloat | char_lowercase | char_frominteger | char_isdigit | char_isletter | char_iswhitespace | char_uppercase
end

In [ ]:
flattened_instruction_sets["gtm.clj"]

In [ ]:
function init_gtm(push_state)
    push_state[:gtm] = Dict(:position => 0, "delay" => 0, "tracks" => [Dict() for i in 1:3], "trace" => [])
    return push_state
end

function ensure_instruction_map(instr_map)
    if isa(instr_map, Dict)
        return instr_map
    else
        return Dict(:instruction => "exec_noop", :close => 0, :silent => false, :random_insertion => false, :uuid => UUIDs.uuid4())
    end
end

function load_track(push_state, track_index, genome)
    push_state['gtm']['tracks'][track_index] = Dict{Int, Dict}()
    for i in 0:length(genome)-1
        push_state['gtm']['tracks'][track_index][i] = ensure_instruction_map(genome[i+1])
    end
    return push_state
end

function dump_track(push_state, track_index)
    gtm = push_state.gtm
    track = gtm.tracks[track_index]
    return [ track[i] for i in eachindex(track) if track[i] != nothing ]
end

function trace(trace_info, push_state)
    return update(push_state, :gtm => :trace => push_state[:gtm][:trace] + [trace_info])
end

init_gtm(Dict())

In [ ]:
ex = :(println())

ex.args